# Book recommendation notebook for Model 3

This notebook contains the recommendation algorithm made for model for the book exchange app.

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
# import books dataset
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/book_recommendations/model_2_data_updated.csv',encoding='latin-1')

In [41]:
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
0,6FpshRoVke3hae7sgf9SQh,Risk Management and Financial Institutions: Ed...,John C. Hull,John Wiley & Sons,9.781119e+12,English,5,3.0
1,JNnqEarcuFkpM5wasJDGcS,Analysis of Financial Time Series: Edition 3,Ruey S. Tsay,John Wiley & Sons,9.781118e+12,English,8,5.0
2,SvbqEjRWxbpDSLxwFjemXP,Intermediate Accounting| 16th Edition: Edition 16,Donald E. Kieso,Wiley Global Education,9.781119e+12,English,1,4.4
3,edMVSPFKsqPSH3J8NG9tan,Key Account Plans,Lynette Ryals,Routledge,9.781136e+12,English,1,0.0
4,RBAFZVjrEWSgkh6TEA5chT,Accounting: Tools for Business Decision Making...,Paul D. Kimmel,Wiley Global Education,9.781119e+12,English,6,5.0


In [42]:
df.shape

(4501, 8)

In [43]:
# unique user id's in the dataset
df['user_id'].unique()

array(['6FpshRoVke3hae7sgf9SQh', 'JNnqEarcuFkpM5wasJDGcS',
       'SvbqEjRWxbpDSLxwFjemXP', ..., 'exUejiwY2q6LK8mS9DZv7p',
       'MYTErjEBA9gVs2qM8DSP7J', 'ZnxGcDjULnyEtVAYPun6yS'], dtype=object)

In [44]:
len(df['user_id'].unique())

4501

In [45]:
df = df.sample(frac = 1)
df.head()

,user_id,title,author,publisher,isbn,language,no_of_exchanges,rating
3696,kQ7djAngwtq4495ZgzqtXW,Ravens in Winter,Bernd Heinrich,Simon and Schuster,9.781477e+12,English,3,4.2
609,mbasMBq882PoBXNJw7zYVg,The Secret Diary of Adrian Mole Aged 13 3/4: A...,Sue Townsend,Penguin UK,9.780718e+12,English,1,4.5
687,Z8FNANoCSqJrYDDoSn4YXD,Fires of Winter,Johanna Lindsey,Harper Collins,9.780062e+12,English,4,4.4
129,9uBe85xSyjWxdZgzdg8zRn,Early Childhood Education: Becoming a Professi...,Kimberly A. Gordon Biddle,SAGE Publications,9.781483e+12,English,0,0.0
1186,UC9K6LvfUVu4HrXDv8tto7,If You Were Mine: The Sullivans,Bella Andre,Oak Press| LLC,9.781938e+12,English,0,4.3


In [46]:
df.dtypes

user_id              object
 title               object
 author              object
 publisher           object
 isbn               float64
 language            object
 no_of_exchanges      int64
 rating             float64
dtype: object

In [47]:
df.isna().sum()

user_id               0
 title                0
 author             109
 publisher            0
 isbn               336
 language             1
 no_of_exchanges      0
 rating               0
dtype: int64

In [48]:
df.dropna(inplace=True)

In [49]:
df.isna().sum()

user_id             0
 title              0
 author             0
 publisher          0
 isbn               0
 language           0
 no_of_exchanges    0
 rating             0
dtype: int64

In [50]:
from scipy.sparse import csr_matrix
# pivot rating into movie features
mat = df.pivot(index=' isbn',columns='user_id',values=' no_of_exchanges').fillna(0)

In [51]:
mat.head()

user_id,23BJ2drCBSYk7h57n9aQ8k,23REmSipgFSEJCQbHqywby,23cvGXWbKmwa4zpjnajfqD,23kB6dgKeDoypvr2UGzcLS,24ufZoymG7wLGKXZp4242H,25PMQ8tkaPqnstu4vXpsJ8,25XSnAeZ2798LFhqazDjqF,26gPMh7KMs2YMuW4w3L4VP,27F3HGCaRMJKEnTmmx5zTW,29aC3w7NLBMPykL2PgRjhX,2BFdWCpytbM4oRBWxXiZfP,2CBofYJRrfLEJugjnAz3s3,2CU3KPA24asyMj5ACMePJg,2CuYJHuAfk4mATSmaiQFF2,2EAjsG6dqAfD5Vmv5EdihF,2EEtCnQQYTRtvFzuWhNzik,2Et73aFmJM7zGZiRgicutR,2Ew6KzzxzT6BTMaLvmW9sy,2FNYDzHGhEE9YgLVD7WDcb,2FuviF2s7WXarTL9H9SLPZ,2GDwfEPaTSPuKut82pbEdT,2GS2DoAWhaDET7rWDN445F,2Gr5r6H59oHciFDaq8bHqE,2H2zzmUJtFZ2vQXkVfT7dC,2HDSPpWuxMvPoMTVYzG9YD,2Hsg2CN3gBkczMFuf9eDSE,2JLZEWe7XPw9zyqE5P5VC2,2JTd4U7E6eXuhuz62vuiZA,2Jx5EWPEUVmFxuQyDATMz6,2JzgrBwRJBeZLsbzxWoJGW,2LLEeydkzDXwR8eNBhRvac,2LzoBJ8zQmNjGUZqdFjrxE,2New7ngYHFierZzPJdk87R,2NwGweCuQq97Ho6A4EtAUJ,2PRpfm9s6fME5H2ZdjS8PC,2PiXWP5KuwaVgQexwiKn9P,2Pz97fGMe3qzWpbmTLsugm,2QZwrzkvYKAhSE3Ed85Z5P,2QzUYujuxA9RoSR7VSeFuy,2QzZjtN9G7yNWLysP7Mkiy,...,nzd9tWKNeDSo8rg6TAPKoo,nzvA5hbb2atUik29M2FWLy,o2mp3VqcwBC7GoSemyPWBM,o325no24SpNo5FgTjXeESB,o48q7MJctqfrctQUqPtwkk,o4A3hhYBbSUn4TwqvsE8e4,o4U3dkdnh3bPb66J5DNFt7,o4VAxwdUNJXjbUo4ge5cTq,o4VuxMebbn7YUqqtqLndfC,o786MfTL4JRQ3BzHTT7Vej,o8E6AbWWxAQvbejnLxiUcm,o8oHHKkPBKtK9y7KCddtDy,o8zAF772tvYPQgqWbQNH8a,oAQu2R9srqmBwsay3myDSD,oB59GNgMhbU2LAmhLmnS4T,oBxPe8YjWeSrGEybobSsgt,oEU7HSEy3QH7BJWsamxzdJ,oGbB4WaDbuubfagWKVi9a2,oHBkGsAfKMtgTaejtHCH6S,oHDTSu2GMMhczYA87UEwgR,oJ8nuZKKKXDfEJGbAMTRit,oKTexmrDZKYTk8BsFS45Mh,oKgRUFN6piQK9Tt9mmrbRr,oKtuEEibkUsGPZyX86v9LK,oL7DBGh52Q3CpwwRjGepCV,oLR7qNamB9X4RGdqTnVf4c,oLVwxCUyCVgWHpK942X3VR,oLohwroBcTSZtrsPaKkSS7,oLyUvuxXxYh9vaWVJw36rH,oN2zuLjPcdYHMWspvcipeb,oRnTtTZWabnPLBDjZjiMBG,oT9KMhb8bemXi7fKGQiBu5,oV22k5v7UNs32hPyx7KtB7,oWHAEhSteE6u7mAf7HdMk5,oXAsdammApn3sch8RzTcS2,oXdtVBkT42nKiCapBHA944,oYC4ETHHrkh8SXvebDYgaR,oYPACtN6pQKhHU4HLPeDYm,oYyNEMgbM739JP4ohqhjx5,oZCFcMW7HCu5RA9riQAiMy
isbn,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9.780007e+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
mat.shape

(3495, 4059)

In [53]:
sparse_mat = csr_matrix(mat.values)

In [54]:
book_map = df[[' isbn', ' title']]

In [55]:
book_map.head()

,isbn,title
3696,9.781477e+12,Ravens in Winter
609,9.780718e+12,The Secret Diary of Adrian Mole Aged 13 3/4: A...
687,9.780062e+12,Fires of Winter
129,9.781483e+12,Early Childhood Education: Becoming a Professi...
1186,9.781938e+12,If You Were Mine: The Sullivans


In [56]:
book_map

,isbn,title
3696,9.781477e+12,Ravens in Winter
609,9.780718e+12,The Secret Diary of Adrian Mole Aged 13 3/4: A...
687,9.780062e+12,Fires of Winter
129,9.781483e+12,Early Childhood Education: Becoming a Professi...
1186,9.781938e+12,If You Were Mine: The Sullivans
...,...,...
755,9.780298e+12,Gone Girl
4352,9.780808e+12,Running Steel| Running America: Race| Economic...
3115,9.781496e+12,The Jerusalem Assassin: A Marcus Ryker Series ...
3018,9.781251e+12,Captivated


In [57]:
book_map.values

array([[9781476794570.0, 'Ravens in Winter'],
       [9780718196134.0,
        'The Secret Diary of Adrian Mole Aged 13 3/4: Adrian Mole Book 1'],
       [9780062106483.0, 'Fires of Winter'],
       ...,
       [9781496437877.0,
        'The Jerusalem Assassin: A Marcus Ryker Series Political and Military Action Thriller: (Book 3)'],
       [9781250775306.0, 'Captivated'],
       [9781988700571.0, 'Anchors Away and Murder']], dtype=object)

In [58]:
hash_map = {
    movie: i for i,movie in enumerate(list(book_map.set_index(' isbn').loc[mat.index][' title']))
}

In [59]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You: The emotional| heartbreaking romantic fiction book from the number one best selling author of Postscript': 4,
 'Love Rosie': 8,
 'Thanks for the Memories': 9,
 'Lords of the Bow (Conqueror| Book 2)': 10,
 'HMS Ulysses': 12,
 'San Andreas': 13,
 'Temeraire (The Temeraire Series| Book 1)': 15,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 16,
 'The Return of the King (The Lord of the Rings| Book 3)': 17,
 'Unfinished Tales': 18,
 'The Children of Húrin': 19,
 'The Lord of the Rings: The classic fantasy masterpiece': 21,
 'The Pagan Lord (The Last Kingdom Series| Book 7)': 22,
 'Mere Christianity': 24,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 25,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 26,
 'Remarkable Creatures': 27,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 28,
 'The Demon King (The Seven Realms Series| Book 1)': 29,
 'The Homecoming of Samuel Lake': 30,

In [60]:
!pip install fuzzywuzzy

In [61]:
# function for searching books in database using keywords
from fuzzywuzzy import fuzz 
def matching(book_map, fav_book):
  """
  returns the closest match via fuzzy ratio
  If no match found, returns None

  Parameters
  --------
  book_map : dataframe of book titles along with their isbn
  fav_book : name of the favourite book

  Return
  -------
  index of closest match
  """
  match = []
  # get match
  for title,isbn in book_map.items():
    ratio = fuzz.ratio(title.lower(),fav_book.lower())
    if ratio >=60:
      match.append((title,isbn, ratio))
    # sort
  match = sorted(match,key = lambda x:x[2])[::-1]
  if not match:
    print("No match found!!!")
  else:
    print("Found possible matches in our database: {0}".format([x[0] for x in match]))
    return match[0][1]

In [62]:
matching(hash_map,"Requiem for Dream")

Found possible matches in our database: ['Requiem for a Mezzo', 'Requiem for a Dream: A Novel']


2521

In [63]:
# function for making recommendations
import time
from sklearn.neighbors import NearestNeighbors
def recommend(fav_book,n_recommendations,book_map,data):
  """
  Returns the top n recommendations based on the keyword entered

  Parameters
  ---------
  fav_book : Keyword for favourite book
  n_recommendations : Number of recommendations to be made
  book_map : book database with title and isbn
  data : csr matrix of user_id, ratings and isbn

  Return
  --------
  list of top n book recommendations
  """
  # instantiating model
  model = NearestNeighbors(algorithm ='brute',metric='cosine')
  # fitting the model
  model.fit(data)
  # get input book isbn
  book_isbn = matching(book_map, fav_book)
  # inference
  print("Recommendation system starting to make inference......")
  t0 = time.time()
  distances, isbns = model.kneighbors(
      data[book_isbn],
      n_neighbors=n_recommendations+1
  )
  recommends = sorted(list(zip(isbns.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  print(f"Time Taken for recommendation : {time.time()-t0}")
  return recommends 

In [64]:
# function to show movie names after recommedations
def make_recommendations(fav_book, n_recommendations, mat, hash_map):
  """
  makes n recommendations based on fav books
  Parameters
  ----------
  fav_book : favourite book
  n_recommendation : number of recommedations to be made
  Return
  ----------
  None, print recommendations
  """
  raw_recommendations = recommend(fav_book,n_recommendations=10,book_map=hash_map, data=mat)
  reverse_hashmap = {v : k for k,v in hash_map.items}
  print('Recommendations for {}:'.format(fav_book))
  for i,(isbn,dist) in enumerate(raw_recommendations):
    print('{0}: {1}, with distance of {2}'.format(i+1,reverse_hashmap[isbn],dist))

In [65]:
book_map

,isbn,title
3696,9.781477e+12,Ravens in Winter
609,9.780718e+12,The Secret Diary of Adrian Mole Aged 13 3/4: A...
687,9.780062e+12,Fires of Winter
129,9.781483e+12,Early Childhood Education: Becoming a Professi...
1186,9.781938e+12,If You Were Mine: The Sullivans
...,...,...
755,9.780298e+12,Gone Girl
4352,9.780808e+12,Running Steel| Running America: Race| Economic...
3115,9.781496e+12,The Jerusalem Assassin: A Marcus Ryker Series ...
3018,9.781251e+12,Captivated


In [66]:
book_map[book_map[' isbn']==9781407097626]

,isbn,title
854,9.781407e+12,The Lost World


In [67]:
print(sparse_mat[1])

  (0, 342)	6.0


In [68]:
hash_map

{'Half of a Yellow Sun': 3,
 'PS| I Love You: The emotional| heartbreaking romantic fiction book from the number one best selling author of Postscript': 4,
 'Love Rosie': 8,
 'Thanks for the Memories': 9,
 'Lords of the Bow (Conqueror| Book 2)': 10,
 'HMS Ulysses': 12,
 'San Andreas': 13,
 'Temeraire (The Temeraire Series| Book 1)': 15,
 'The Fellowship of the Ring (The Lord of the Rings| Book 1)': 16,
 'The Return of the King (The Lord of the Rings| Book 3)': 17,
 'Unfinished Tales': 18,
 'The Children of Húrin': 19,
 'The Lord of the Rings: The classic fantasy masterpiece': 21,
 'The Pagan Lord (The Last Kingdom Series| Book 7)': 22,
 'Mere Christianity': 24,
 'Sharpe\x92s Tiger: The Siege of Seringapatam| 1799 (The Sharpe Series| Book 1)': 25,
 'The Last Kingdom (The Last Kingdom Series| Book 1)': 26,
 'Remarkable Creatures': 27,
 'A Foreign Country (Thomas Kell Spy Thriller| Book 1)': 28,
 'The Demon King (The Seven Realms Series| Book 1)': 29,
 'The Homecoming of Samuel Lake': 30,

In [69]:
len(hash_map)

3460

In [70]:
book_dict = dict(zip(book_map[' isbn'],book_map[' title']))

In [71]:
# function to get list of recommmendations based on isbn
def recommendations_from_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  This function provides recommendations based on isbn
  
  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  recoms: recommendations from the passed isbn
  """
  dummy_model = NearestNeighbors(algorithm='brute',metric='cosine')
  dummy_model.fit(data)
  title = book_dict[isbn]
  r,d = dummy_model.kneighbors(data[hash_map[title]],n_neighbors=11)
  recoms = []
  recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  for (i,j) in recommends:
    for title, isbn in hash_map.items():
      if isbn == i:
        recoms.append(title)
  return recoms

In [72]:
recommendations_from_isbn(9780984835935.0,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

['Wool',
 'The Wolf Gift',
 "War of Don Emmanuel's Nether Parts",
 'Casino Royale',
 'The Name of the Rose']

In [73]:
# function to get recommended books' isbn
def get_books_isbn(isbn,data,n_neighbors,book_dict,hash_map):
  """
  Function to return isbns of recommended books from the isbn passed

  Parameters
  ---------
  isbn: isbn of the book from which recommendations have to be made
  data: sparse matrix data
  n_neighbors: number of neighbors for knn models
  book_dict: dictionary linking isbn to book name
  hash_map: dictionary linking title to sparse matrix index

  Return
  --------
  isbns: isbns of recommendations from the passed isbn

  """
  recoms = recommendations_from_isbn(isbn,data=data,n_neighbors=n_neighbors,book_dict=book_dict,hash_map=hash_map)
  isbns = []
  for book_name in recoms:
    for isb,title in book_dict.items():
      if title == book_name:
        isbns.append(isb)
  return isbns

In [74]:
get_books_isbn(9780748117352.0,data=sparse_mat,n_neighbors=10, book_dict=book_dict, hash_map=hash_map)

[9781448139859.0,
 9781448113798.0,
 9781448137749.0,
 9781448139248.0,
 9781448136285.0]

In [77]:
# Making recommendations using only one base model
base_model = NearestNeighbors(algorithm='brute', metric='cosine')
base_model.fit(sparse_mat)
title = book_dict[9781448139859.0]
r,d = base_model.kneighbors(sparse_mat[hash_map[title]],n_neighbors=11)
recoms = []
recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
for (i,j) in recommends:
  for title, isbn in hash_map.items():
    if isbn == i:
      recoms.append(title)
recoms

['Good Omens',
 "War of Don Emmanuel's Nether Parts",
 "The Orphan Master's Son: Barack Obama\x92s Summer Reading Pick 2019",
 'The Name of the Rose',
 'The Wolf Gift',
 'Star Wars: Darth Plagueis']

In [79]:
isbns = []
for book_name in recoms:
  for isb,title in book_dict.items():
    if title == book_name:
      isbns.append(isb)
isbns

[9781448110230.0,
 9781448113798.0,
 9781448109265.0,
 9781448137749.0,
 9781448135318.0,
 9781448136285.0]

In [80]:
# Making a base model and training it
model = NearestNeighbors(metric='cosine',algorithm='brute')
model.fit(sparse_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [85]:
# Function for getting book title using base model
def get_title(model,isbn):
  """
  Function to return title of recommended books using isbn

  Parameters
  -----------
  model : base model globally trained to recommend books
  isbn : isbn of book from which recommendations are to be made

  Return
  -----------
  recoms : list of titles of recommended books
  """
  title = book_dict[isbn]
  r,d = base_model.kneighbors(sparse_mat[hash_map[title]],n_neighbors=11)
  recoms = []
  recommends = sorted(list(zip(d.squeeze().tolist(),r.squeeze().tolist())),key=lambda x : x[1])[:0:-1]
  for (i,j) in recommends:
    for title, isbn in hash_map.items():
      if isbn == i:
        recoms.append(title)
  return recoms

In [88]:
get_title(model,isbn=9781448139859.0)

['Good Omens',
 "War of Don Emmanuel's Nether Parts",
 "The Orphan Master's Son: Barack Obama\x92s Summer Reading Pick 2019",
 'The Name of the Rose',
 'The Wolf Gift',
 'Star Wars: Darth Plagueis']

In [86]:
# Function to get isbn from title
def get_recommendations(model,isbn):
  """
  Function to isbn of recommended books by using a globally declared model

  Parameters
  ----------
  model - globally trained model for recommendations
  isbn - isbn of book from which recommendations are to be made

  Return
  ----------
  isbns - list of recommended books' isbns
  """
  recoms = get_title(model,isbn)
  isbns = []
  for book_name in recoms:
    for isb,title in book_dict.items():
      if title == book_name:
        isbns.append(isb)
  return isbns

In [87]:
get_recommendations(model,isbn=9781448139859.0)

[9781448110230.0,
 9781448113798.0,
 9781448109265.0,
 9781448137749.0,
 9781448135318.0,
 9781448136285.0]

In [89]:
# Export model pkl file
import pickle as pkl
pkl.dump(model,open('/content/drive/MyDrive/book_recommendations/model_file.pkl','wb'))